# 수익률 PEIS점수테이블에 붙이기 

In [1]:
import pandas as pd
import numpy as np
import warnings
from Fin import *
warnings.filterwarnings(action='ignore')

# 0. 파일불러들이기 붙여질 파일(PEIS) + 붙일 파일(price)
# 목표 : 주가랑 주가상승률, 애널리스트등급 붙이기

In [2]:
PEIS_table = pd.read_excel("Data/5_4_data10.xlsx",sheet_name = "kospi_peis")
PROFIT_table = pd.read_excel("Data/5_4_data10.xlsx",sheet_name = "kospi_stock")

In [13]:
PROFIT_table.drop(['Item','Frequency','Kind'],axis=1,inplace=True)

# 데이터 전처리 
# 1. 주가랑 주가상승률, 애널리스트등급 행렬로 일자로 만들기

In [14]:
PROFIT_table = MakeLookGood(PROFIT_table, years = list(range(10,22)))
Onetable = MakeOnetable(PROFIT_table, ['수정주가(원)','투자의견점수 (E3)(포인트)'], ['ADJprice', 'Consensus'])
Onetable['deltaprice'] = Caldeltaprice(Onetable['ADJprice'])
# 여기서 deltaprice 수정사항 넣어야함
Onetable = Onetable.fillna(99)  

# 2. 붙이기

In [15]:
PEIS_table = pd.concat([PEIS_table,Onetable],axis=1)

# 3. 저장하기

In [16]:
# 이렇게 만든거는.. 일단 주가 정보가 없는 친구들도 있어서 얘들은 포트폴리오로 구성하기가 힘들 듯함 상장전인듯
# 그리고 중간에 상폐당한 친구들은 또 PEIS는 있는데 주가가 없어서 포폴게산이 안될떄가 있을 거 같음 우짬.. 
PEIS_table.to_excel("Data/KospiPEISData&PriceData.xlsx")

# 남은 일 코스닥 코스피 아무렇게 불러서 붙여지도록 자동화